<a href="https://colab.research.google.com/github/pnn99/detector_fraude_LLM/blob/main/detector_de_fraude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud?resource=download


In [ ]:
# !pip install pandas
# !pip install numpy
# !pip install sklearn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

In [ ]:
df_credit = pd.read_csv('creditcard.csv')
df_credit.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
#verificação de tipos e valores nulos
df_credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
#verifica os tipos dos dados não fraude
df_nao_fraude = df_credit.Amount[df_credit.Class == 0]
df_nao_fraude.describe()

,Amount
count,284315.000000
mean,88.291022
std,250.105092
min,0.000000
25%,5.650000
50%,22.000000
75%,77.050000
max,25691.160000


In [ ]:
#verifica os tipos dos dados de fraude
df_fraude = df_credit.Amount[df_credit.Class == 1]
df_fraude.describe()

,Amount
count,492.000000
mean,122.211321
std,256.683288
min,0.000000
25%,1.000000
50%,9.250000
75%,105.890000
max,2125.870000


In [ ]:
# Contagem das classes (fraude e não fraude)
df_credit.Class.value_counts()

,count
Class,
0,284315
1,492


In [ ]:
#Criação de um novo dataset para utilizar um undersampling para igualar a nossa base de dados
df_fraude = df_credit[df_credit.Class == 1]
df_nao_fraude = df_credit[df_credit.Class == 0]

df_undersampling = df_nao_fraude.sample(n=492)
df_undersampling

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
140661,83855.0,-0.687175,-0.240117,1.058203,-2.427235,-1.201069,-0.400761,-0.423202,0.473449,-2.867547,...,0.077734,0.321270,0.159450,0.183462,-0.431339,-0.418307,0.016044,0.070051,63.50,0
107060,70264.0,-0.623904,1.089138,1.228114,-0.184476,0.045576,-0.677563,0.582286,0.046214,0.014123,...,-0.303941,-0.808396,-0.055866,-0.170576,-0.115669,0.110777,0.360201,0.180240,2.58,0
93296,64341.0,1.110928,-0.145158,1.391820,1.239423,-1.051184,0.140883,-0.905013,0.317634,0.856572,...,0.264241,0.783500,-0.022473,0.041825,0.240140,-0.204064,0.082181,0.037493,9.99,0
103519,68694.0,-0.275183,0.244558,0.547084,0.509140,0.561911,-0.179794,0.297584,0.026737,0.417548,...,-0.120678,0.266320,0.340635,-0.061195,-0.328789,0.273325,-0.005176,-0.250293,12.31,0
222332,142932.0,-1.478828,1.374160,0.477209,-0.901273,-0.319597,-0.628338,0.163575,0.635432,-0.206945,...,0.099917,0.421986,-0.104275,0.130594,-0.184195,0.489154,-0.110413,-0.106183,11.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80259,58401.0,1.258239,0.341060,0.300209,0.693661,-0.388141,-1.075821,0.070370,-0.190626,0.066662,...,-0.291059,-0.848659,0.132151,0.333675,0.209611,0.095588,-0.024365,0.030135,0.89,0
148731,90157.0,-0.208245,0.521945,-0.363266,-0.061798,1.303316,-0.803370,1.479262,-0.621777,1.147050,...,0.183736,0.821110,-0.041032,-0.836862,-0.355597,-0.189175,-0.008244,0.022492,127.69,0
41477,40698.0,1.290130,-0.747392,0.970566,-0.685321,-1.539362,-0.625913,-1.014584,0.089360,-0.641903,...,0.187240,0.334649,0.052889,0.526212,0.171639,-0.327038,0.017246,0.020078,28.02,0
23,18.0,0.247491,0.277666,1.185471,-0.092603,-1.314394,-0.150116,-0.946365,-1.617935,1.544071,...,1.650180,0.200454,-0.185353,0.423073,0.820591,-0.227632,0.336634,0.250475,22.75,0


In [ ]:
#concaternar dos dataframes
df_concatenado = pd.concat([df_undersampling, df_fraude], axis=0)
df_concatenado

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
140661,83855.0,-0.687175,-0.240117,1.058203,-2.427235,-1.201069,-0.400761,-0.423202,0.473449,-2.867547,...,0.077734,0.321270,0.159450,0.183462,-0.431339,-0.418307,0.016044,0.070051,63.50,0
107060,70264.0,-0.623904,1.089138,1.228114,-0.184476,0.045576,-0.677563,0.582286,0.046214,0.014123,...,-0.303941,-0.808396,-0.055866,-0.170576,-0.115669,0.110777,0.360201,0.180240,2.58,0
93296,64341.0,1.110928,-0.145158,1.391820,1.239423,-1.051184,0.140883,-0.905013,0.317634,0.856572,...,0.264241,0.783500,-0.022473,0.041825,0.240140,-0.204064,0.082181,0.037493,9.99,0
103519,68694.0,-0.275183,0.244558,0.547084,0.509140,0.561911,-0.179794,0.297584,0.026737,0.417548,...,-0.120678,0.266320,0.340635,-0.061195,-0.328789,0.273325,-0.005176,-0.250293,12.31,0
222332,142932.0,-1.478828,1.374160,0.477209,-0.901273,-0.319597,-0.628338,0.163575,0.635432,-0.206945,...,0.099917,0.421986,-0.104275,0.130594,-0.184195,0.489154,-0.110413,-0.106183,11.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [ ]:
#ajustar o Index
df_concatenado.reset_index(drop=True, inplace=True)
df_concatenado

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,83855.0,-0.687175,-0.240117,1.058203,-2.427235,-1.201069,-0.400761,-0.423202,0.473449,-2.867547,...,0.077734,0.321270,0.159450,0.183462,-0.431339,-0.418307,0.016044,0.070051,63.50,0
1,70264.0,-0.623904,1.089138,1.228114,-0.184476,0.045576,-0.677563,0.582286,0.046214,0.014123,...,-0.303941,-0.808396,-0.055866,-0.170576,-0.115669,0.110777,0.360201,0.180240,2.58,0
2,64341.0,1.110928,-0.145158,1.391820,1.239423,-1.051184,0.140883,-0.905013,0.317634,0.856572,...,0.264241,0.783500,-0.022473,0.041825,0.240140,-0.204064,0.082181,0.037493,9.99,0
3,68694.0,-0.275183,0.244558,0.547084,0.509140,0.561911,-0.179794,0.297584,0.026737,0.417548,...,-0.120678,0.266320,0.340635,-0.061195,-0.328789,0.273325,-0.005176,-0.250293,12.31,0
4,142932.0,-1.478828,1.374160,0.477209,-0.901273,-0.319597,-0.628338,0.163575,0.635432,-0.206945,...,0.099917,0.421986,-0.104275,0.130594,-0.184195,0.489154,-0.110413,-0.106183,11.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
980,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
981,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
982,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [ ]:
#retirar dados para validação
df_val_nao_fraude = df_concatenado.head(5)
df_val_nao_fraude

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,83855.0,-0.687175,-0.240117,1.058203,-2.427235,-1.201069,-0.400761,-0.423202,0.473449,-2.867547,...,0.077734,0.321270,0.159450,0.183462,-0.431339,-0.418307,0.016044,0.070051,63.50,0
1,70264.0,-0.623904,1.089138,1.228114,-0.184476,0.045576,-0.677563,0.582286,0.046214,0.014123,...,-0.303941,-0.808396,-0.055866,-0.170576,-0.115669,0.110777,0.360201,0.180240,2.58,0
2,64341.0,1.110928,-0.145158,1.391820,1.239423,-1.051184,0.140883,-0.905013,0.317634,0.856572,...,0.264241,0.783500,-0.022473,0.041825,0.240140,-0.204064,0.082181,0.037493,9.99,0
3,68694.0,-0.275183,0.244558,0.547084,0.509140,0.561911,-0.179794,0.297584,0.026737,0.417548,...,-0.120678,0.266320,0.340635,-0.061195,-0.328789,0.273325,-0.005176,-0.250293,12.31,0
4,142932.0,-1.478828,1.374160,0.477209,-0.901273,-0.319597,-0.628338,0.163575,0.635432,-0.206945,...,0.099917,0.421986,-0.104275,0.130594,-0.184195,0.489154,-0.110413,-0.106183,11.00,0


In [ ]:
df_val_fraude = df_concatenado.tail(5)
df_val_fraude

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
979,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
980,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
981,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
982,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
983,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,...,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


In [ ]:
df_concatenado.drop(df_concatenado.head(5).index, inplace=True)
df_concatenado.drop(df_concatenado.tail(5).index, inplace=True)
df_concatenado

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
5,19051.0,1.138558,-0.344476,1.125137,-0.635452,-0.991297,-0.096130,-0.831054,0.154802,2.821318,...,-0.066173,0.293136,0.012710,0.188676,0.346499,-0.726301,0.079074,0.018445,6.70,0
6,33159.0,-1.039458,0.426780,1.669247,0.290706,0.837084,-0.028907,0.674719,0.201995,-0.618314,...,0.094760,0.238819,-0.124499,0.022825,0.307672,-0.486444,0.080445,0.102774,42.00,0
7,50190.0,-1.194503,0.958612,1.577105,1.039757,-0.703314,0.260833,0.359605,0.700249,-0.165410,...,-0.157604,-0.522739,0.088699,0.009061,0.318285,-0.359103,0.190140,0.081198,98.40,0
8,144734.0,1.651251,-2.109132,-0.155290,-0.609264,-1.472363,1.100069,-1.531595,0.380186,0.826578,...,0.283827,0.606382,0.003639,0.275276,-0.367113,-0.222952,0.018536,-0.005879,230.34,0
9,76213.0,-0.964515,0.511409,1.360282,0.764367,0.527291,0.137962,0.541090,0.117335,-0.362465,...,-0.011380,0.270700,0.116472,0.032397,0.267420,-0.375967,-0.113411,-0.041111,14.31,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,165981.0,-5.766879,-8.402154,0.056543,6.950983,9.880564,-5.773192,-5.748879,0.721743,-1.076274,...,0.880395,-0.130436,2.241471,0.665346,-1.890041,-0.120803,0.073269,0.583799,0.00,1
975,166028.0,-0.956390,2.361594,-3.171195,1.970759,0.474761,-1.902598,-0.055178,0.277831,-1.745854,...,0.473211,0.719400,0.122458,-0.255650,-0.619259,-0.484280,0.683535,0.443299,39.90,1
976,166831.0,-2.027135,-1.131890,-1.135194,1.086963,-0.010547,0.423797,3.790880,-1.155595,-0.063434,...,-0.315105,0.575520,0.490842,0.756502,-0.142685,-0.602777,0.508712,-0.091646,634.30,1
977,166883.0,2.091900,-0.757459,-1.192258,-0.755458,-0.620324,-0.322077,-1.082511,0.117200,-0.140927,...,0.288253,0.831939,0.142007,0.592615,-0.196143,-0.136676,0.020182,-0.015470,19.95,1


In [ ]:
#concatenar o dataframe de validação
df_val_total = pd.concat([df_val_nao_fraude, df_val_fraude], axis=0)
df_val_total.reset_index(inplace=True)

df_val_total_real = df_val_total.Class
df_val_total = df_val_total.drop(['index','Time','Class'], axis=1)
df_val_total

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-0.687175,-0.240117,1.058203,-2.427235,-1.201069,-0.400761,-0.423202,0.473449,-2.867547,0.677693,...,-0.380544,0.077734,0.321270,0.159450,0.183462,-0.431339,-0.418307,0.016044,0.070051,63.50
1,-0.623904,1.089138,1.228114,-0.184476,0.045576,-0.677563,0.582286,0.046214,0.014123,0.035682,...,0.166636,-0.303941,-0.808396,-0.055866,-0.170576,-0.115669,0.110777,0.360201,0.180240,2.58
2,1.110928,-0.145158,1.391820,1.239423,-1.051184,0.140883,-0.905013,0.317634,0.856572,-0.007241,...,-0.227348,0.264241,0.783500,-0.022473,0.041825,0.240140,-0.204064,0.082181,0.037493,9.99
3,-0.275183,0.244558,0.547084,0.509140,0.561911,-0.179794,0.297584,0.026737,0.417548,-0.321059,...,0.034390,-0.120678,0.266320,0.340635,-0.061195,-0.328789,0.273325,-0.005176,-0.250293,12.31
4,-1.478828,1.374160,0.477209,-0.901273,-0.319597,-0.628338,0.163575,0.635432,-0.206945,-0.271106,...,0.015743,0.099917,0.421986,-0.104275,0.130594,-0.184195,0.489154,-0.110413,-0.106183,11.00
5,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,-5.587794,...,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00
6,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,-3.232153,...,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76
7,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,-3.463891,...,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89
8,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,-5.245984,...,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00
9,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,-0.888722,...,-0.017652,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53


In [ ]:
#verificar a distribuição dos dados fraude/não fraude
df_concatenado.Class.value_counts()

,count
Class,
0,487
1,487


In [ ]:
#separar labels e features
X = df_concatenado.drop(['Time','Class'], axis=1)
Y = df_concatenado['Class']

In [ ]:
print(X, Y)

           V1        V2        V3        V4        V5        V6        V7  \
5    1.138558 -0.344476  1.125137 -0.635452 -0.991297 -0.096130 -0.831054   
6   -1.039458  0.426780  1.669247  0.290706  0.837084 -0.028907  0.674719   
7   -1.194503  0.958612  1.577105  1.039757 -0.703314  0.260833  0.359605   
8    1.651251 -2.109132 -0.155290 -0.609264 -1.472363  1.100069 -1.531595   
9   -0.964515  0.511409  1.360282  0.764367  0.527291  0.137962  0.541090   
..        ...       ...       ...       ...       ...       ...       ...   
974 -5.766879 -8.402154  0.056543  6.950983  9.880564 -5.773192 -5.748879   
975 -0.956390  2.361594 -3.171195  1.970759  0.474761 -1.902598 -0.055178   
976 -2.027135 -1.131890 -1.135194  1.086963 -0.010547  0.423797  3.790880   
977  2.091900 -0.757459 -1.192258 -0.755458 -0.620324 -0.322077 -1.082511   
978 -1.374424  2.793185 -4.346572  2.400731 -1.688433  0.111136 -0.922038   

           V8        V9       V10  ...       V20       V21       V22  \
5  

In [ ]:
#separar entre daddos de treino e teste
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state = 42)

In [ ]:
#Treinamento

lr = LogisticRegression(max_iter=1000)
lr.fit(x_train, y_train)
pred = lr.predict(x_test)
acc = accuracy_score(y_test, pred)

f'Precisão do modelo: {acc * 100:.2f}'

'Precisão do modelo: 95.38'

In [ ]:
#validação

pred = lr.predict(df_val_total)

df = pd.DataFrame({'Real': df_val_total_real, 'Previsão': pred})
df

,Real,Previsão
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,1,1
6,1,1
7,1,1
8,1,1
9,1,0
